GPU上での高速な物理シミュレーションは、(RLHFやOffline RLに押され気味とはいえ)強化学習界隈では話題のトピックですよね。また単純に、GPU上で爆速でシミュレーションが終わるのはなかなか楽しいものです。
[NVIDIA IsaacSym](https://docs.omniverse.nvidia.com/index.html)もありますが、jaxで強化学習パイプライン全体を高速化したいなら[brax](https://github.com/google/brax)が便利です。以前[紹介するブログ](https://kngwyu.github.io/rlog2/posts/jax-brax-haiku.html)も書きましたが、現在のバージョンではより精度のいい手法が選べるようになっていて、普通にMuJoCoの代わりに使えそうな感じです。しかし、最近単純な2次元物理シミュレーションでbraxが使えないかな？と思って検討してみたところ、無理ではないのだけれどどうにも使いづらいな...という印象でした。また、二次元物理シミュレーションをするのに、三次元のボールとかで当たり判定を行うのはちょっと計算資源がもったいない気もします。なら自分で作ってしまえばいいんじゃないか？ということでやってみました。

# 2次元ゲーム物理？

このブログ記事では、いわゆる「2次元ゲーム物理」の代表格として、[Box2D](https://en.wikipedia.org/wiki/Box2D)や[Chipmunk](tps://github.com/slembcke/Chipmunk2D)で実装されている比較的簡素なインパルスベースのシミュレーションを実装してみます。これは、物体が衝突した後の速度・位置の解決を、「衝突した位置から適当なインパルスを与える」ことで解く手法になります。braxでは

> Spring provides fast and cheap simulation for rapid experimentation, using simple impulse-based methods often found in video games.

と説明されています。関節が多い場合などはちゃんと拘束条件式と運動方程式を解かないとあまり精度が出ないのですが、単純な2次元物理シミュレーションならこれで十分でしょう。

## 何をすればいいの

では、このインパルスベースのシミュレーションでは、どのような流れでシミュレーションを実装すればいいのでしょうか。

1. 現在の速度をもとに剛体を動かし、速度を減衰させる
2. 剛体同士の衝突判定を行う
3. 衝突した物体を交差しない位置まで戻して、衝突と逆方向のインパルスを加える

まあだいたいこんな感じですね。必要に応じて「壁すり抜け対策」（物体が動いた軌道同士の交差判定）も行ったほうがいいですが、MuJoCoなど多くの物理シミュレーターでは$dt$が十分小さいことを仮定して実装されていないので、今回は実装しないことにします。

# とりあえず作ってみる

In [2]:
import chex
import jax
import jax.numpy as jnp


@chex.dataclass
class Circle:
    mass: jax.Array
    radius: jax.Array
    rgba: jax.Array


@chex.dataclass
class Velocity:
    angle_v: jax.Array
    xy_v: jax.Array


@chex.dataclass
class Position:
    angle: jax.Array
    xy: jax.Array


@chex.dataclass
class Space:
    dt: jax.Array
    dynamic_circle: jax.Array
    static_circle: jax.Array